In [1]:
import requests
from lxml import html
import pandas as pd
from bs4 import BeautifulSoup
import re
import sys
from datetime import datetime
import time
import numpy as np
import json
from config import *

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select

import time
import json
import re
from tqdm import tqdm

In [ ]:
#strava

# stats= https://www.strava.com/activities/586018101/streams?stream_types%5B%5D=timer_time&stream_types%5B%5D=altitude&stream_types%5B%5D=heartrate&stream_types%5B%5D=cadence&stream_types%5B%5D=temp&stream_types%5B%5D=distance&stream_types%5B%5D=grade_smooth&stream_types%5B%5D=time&stream_types%5B%5D=grade_adjusted_distance&_=1537842990620
# latlng = https://www.strava.com/activities/586018101/streams?stream_types%5B%5D=timer_time&stream_types%5B%5D=latlng&_=1537842990619

pete = "https://www.strava.com/athletes/16102568/training/log?feature=sidebar-progress-goals-v2"
adam= "https://www.strava.com/pros/11620352#interval?interval=201607&interval_type=week&chart_type=miles&year_offset=2"
jermey = "https://www.strava.com/athletes/1150260#graph_date_range?chart_type=miles&interval_type=week&interval=201640&year_offset=2"
jason = "https://www.strava.com/athletes/13823509#interval?interval=201621&interval_type=week&chart_type=miles&year_offset=2"

In [14]:
#pull date, distance, time, pace

activities = []
links = []
dates = []
html_text = []

        #load webpage
browser = webdriver.Chrome() 
browser.get("https://www.strava.com/login") #navigate to the page
browser.implicitly_wait(5)

            # login = browser.find_element_by_xpath('//*[@id="global-nav"]/div/ul/li[5]/a').click()

#sign-in
username = browser.find_element_by_id('email')
username.clear()
username.send_keys(username_c)

password = browser.find_element_by_id('password')
password.clear()
password.send_keys(strava_pw)

browser.find_element_by_id("login-button").click()

#search
browser.find_element_by_xpath('//*[@id="global-search-field"]').send_keys('jeremy bradford')
browser.implicitly_wait(2)
browser.find_element_by_xpath('//*[@id="global-search-button"]').click()
browser.implicitly_wait(2)
browser.find_element_by_xpath('/html/body/div[1]/div[3]/div/ul/li[1]/div[1]/div[2]/h4/a').click()


#change year of data
browser.get('https://www.strava.com/athletes/1150260#interval_type?chart_type=miles&interval_type=month&interval=201610&year_offset=2')
time.sleep(2)
# browser.find_element_by_xpath('//*[@id="interval-graph-controls"]/ul/li[2]/ul/li[2]/a').click()

# september - click bar graph
for x in range(4,7):
    time.sleep(2)
    browser.find_element_by_xpath(f'//*[@id="interval-20160{str(x)}"]/div/a').click()

    #activity links
    time.sleep(4)

    feed = browser.find_element_by_class_name('feed')
    webdriver.ActionChains(browser).move_to_element(feed)

    html = feed.get_attribute('innerHTML')
    
    html_text.append(html)


In [22]:
#pull name, date, activity stats for each entry on page
#skip if all values aren't present

activity_details = []

for html in html_text:
    bs = BeautifulSoup(html, 'html.parser')
    container = bs.find_all('div', attrs={"class":"entry-container"})
    for a in container:
        try:
            name = a.find_all('a', attrs={'class':"entry-athlete"})[0].get_text().strip('\n').split('\n')[0]
            date = a.find_all('time', attrs={'class':"timestamp"})[0].get_text().strip('\n')
            dist = a.find_all('li', attrs={'title':"Distance"})[0].get_text()
            pace = a.find_all('li', attrs={'title':"Pace"})[0].get_text()
            dur = a.find_all('li', attrs={'title':"Time"})[0].get_text()
            activity_details.append([name, date, dist, pace, dur])
        except:
            pass
activity_details

[['Jeremy Bradford',
  'April 30, 2016 at 9:03 AM',
  '47.73 mi',
  '15:39 /mi',
  '12h 26m'],
 ['Jeremy Bradford',
  'April 29, 2016 at 9:01 PM',
  '4.17 mi',
  '13:40 /mi',
  '57m 0s'],
 ['Jeremy Bradford',
  'April 29, 2016 at 9:00 AM',
  '42.66 mi',
  '12:02 /mi',
  '8h 33m'],
 ['Jeremy Bradford',
  'April 28, 2016 at 10:02 AM',
  '36.34 mi',
  '13:51 /mi',
  '8h 23m'],
 ['Jeremy Bradford',
  'April 27, 2016 at 8:34 AM',
  '50.65 mi',
  '13:26 /mi',
  '11h 20m'],
 ['Jeremy Bradford',
  'April 26, 2016 at 10:20 AM',
  '40.42 mi',
  '13:30 /mi',
  '9h 5m'],
 ['Jeremy Bradford',
  'April 25, 2016 at 7:41 AM',
  '21.07 mi',
  '13:16 /mi',
  '4h 39m'],
 ['Jeremy Bradford',
  'April 23, 2016 at 9:29 PM',
  '34.78 mi',
  '14:57 /mi',
  '8h 40m'],
 ['Jeremy Bradford',
  'April 23, 2016 at 4:30 PM',
  '14.46 mi',
  '12:27 /mi',
  '2h 59m'],
 ['Jeremy Bradford',
  'April 22, 2016 at 5:18 PM',
  '36.24 mi',
  '15:23 /mi',
  '9h 17m'],
 ['Jeremy Bradford',
  'April 22, 2016 at 10:59 AM',
  '14

In [40]:
#convert to dataframe and name columns
jeremy_df = pd.DataFrame(activity_details, columns=['name', 'date', 'distance', 'pace', 'duration'])
jeremy_df

,name,date,distance,pace,duration
0,Jeremy Bradford,"April 30, 2016 at 9:03 AM",47.73 mi,15:39 /mi,12h 26m
1,Jeremy Bradford,"April 29, 2016 at 9:01 PM",4.17 mi,13:40 /mi,57m 0s
2,Jeremy Bradford,"April 29, 2016 at 9:00 AM",42.66 mi,12:02 /mi,8h 33m
3,Jeremy Bradford,"April 28, 2016 at 10:02 AM",36.34 mi,13:51 /mi,8h 23m
4,Jeremy Bradford,"April 27, 2016 at 8:34 AM",50.65 mi,13:26 /mi,11h 20m
5,Jeremy Bradford,"April 26, 2016 at 10:20 AM",40.42 mi,13:30 /mi,9h 5m
6,Jeremy Bradford,"April 25, 2016 at 7:41 AM",21.07 mi,13:16 /mi,4h 39m
7,Jeremy Bradford,"April 23, 2016 at 9:29 PM",34.78 mi,14:57 /mi,8h 40m
8,Jeremy Bradford,"April 23, 2016 at 4:30 PM",14.46 mi,12:27 /mi,2h 59m
9,Jeremy Bradford,"April 22, 2016 at 5:18 PM",36.24 mi,15:23 /mi,9h 17m


In [41]:
#convert date to datetime object
jeremy_df['date'] = jeremy_df['date'].str.replace('at', '')
jeremy_df.date=pd.to_datetime(jeremy_df.date)
jeremy_df

,name,date,distance,pace,duration
0,Jeremy Bradford,2016-04-30 09:03:00,47.73 mi,15:39 /mi,12h 26m
1,Jeremy Bradford,2016-04-29 21:01:00,4.17 mi,13:40 /mi,57m 0s
2,Jeremy Bradford,2016-04-29 09:00:00,42.66 mi,12:02 /mi,8h 33m
3,Jeremy Bradford,2016-04-28 10:02:00,36.34 mi,13:51 /mi,8h 23m
4,Jeremy Bradford,2016-04-27 08:34:00,50.65 mi,13:26 /mi,11h 20m
5,Jeremy Bradford,2016-04-26 10:20:00,40.42 mi,13:30 /mi,9h 5m
6,Jeremy Bradford,2016-04-25 07:41:00,21.07 mi,13:16 /mi,4h 39m
7,Jeremy Bradford,2016-04-23 21:29:00,34.78 mi,14:57 /mi,8h 40m
8,Jeremy Bradford,2016-04-23 16:30:00,14.46 mi,12:27 /mi,2h 59m
9,Jeremy Bradford,2016-04-22 17:18:00,36.24 mi,15:23 /mi,9h 17m


In [42]:
jeremy_df['duration_sum'] =jeremy_df['duration'].str.split(' ')
jeremy_df

,name,date,distance,pace,duration,duration_sum
0,Jeremy Bradford,2016-04-30 09:03:00,47.73 mi,15:39 /mi,12h 26m,"[12h, 26m]"
1,Jeremy Bradford,2016-04-29 21:01:00,4.17 mi,13:40 /mi,57m 0s,"[57m, 0s]"
2,Jeremy Bradford,2016-04-29 09:00:00,42.66 mi,12:02 /mi,8h 33m,"[8h, 33m]"
3,Jeremy Bradford,2016-04-28 10:02:00,36.34 mi,13:51 /mi,8h 23m,"[8h, 23m]"
4,Jeremy Bradford,2016-04-27 08:34:00,50.65 mi,13:26 /mi,11h 20m,"[11h, 20m]"
5,Jeremy Bradford,2016-04-26 10:20:00,40.42 mi,13:30 /mi,9h 5m,"[9h, 5m]"
6,Jeremy Bradford,2016-04-25 07:41:00,21.07 mi,13:16 /mi,4h 39m,"[4h, 39m]"
7,Jeremy Bradford,2016-04-23 21:29:00,34.78 mi,14:57 /mi,8h 40m,"[8h, 40m]"
8,Jeremy Bradford,2016-04-23 16:30:00,14.46 mi,12:27 /mi,2h 59m,"[2h, 59m]"
9,Jeremy Bradford,2016-04-22 17:18:00,36.24 mi,15:23 /mi,9h 17m,"[9h, 17m]"


In [43]:
#create column for duration in minutes
def total_duration(x):
    min_sum = 0
    for item in x:
        min_sum += convert_to_minutes(item)
    return min_sum

def convert_to_minutes(time):
    time_type = time[-1];
    if time_type == "h":
        return int(time[0:-1]) * 60
    if time_type =="m":
        return int(time[0:-1])
    else:
        return 0

jeremy_df['duration_sum'] = jeremy_df['duration_sum'].map(total_duration)
jeremy_df

,name,date,distance,pace,duration,duration_sum
0,Jeremy Bradford,2016-04-30 09:03:00,47.73 mi,15:39 /mi,12h 26m,746
1,Jeremy Bradford,2016-04-29 21:01:00,4.17 mi,13:40 /mi,57m 0s,57
2,Jeremy Bradford,2016-04-29 09:00:00,42.66 mi,12:02 /mi,8h 33m,513
3,Jeremy Bradford,2016-04-28 10:02:00,36.34 mi,13:51 /mi,8h 23m,503
4,Jeremy Bradford,2016-04-27 08:34:00,50.65 mi,13:26 /mi,11h 20m,680
5,Jeremy Bradford,2016-04-26 10:20:00,40.42 mi,13:30 /mi,9h 5m,545
6,Jeremy Bradford,2016-04-25 07:41:00,21.07 mi,13:16 /mi,4h 39m,279
7,Jeremy Bradford,2016-04-23 21:29:00,34.78 mi,14:57 /mi,8h 40m,520
8,Jeremy Bradford,2016-04-23 16:30:00,14.46 mi,12:27 /mi,2h 59m,179
9,Jeremy Bradford,2016-04-22 17:18:00,36.24 mi,15:23 /mi,9h 17m,557


In [44]:
#remove /mi from pace and convert to decimal 
jeremy_df['pace'] =jeremy_df['pace'].str.replace('/mi', '')

def express_pace_decimal(row):
    try:
        minutes_part = float(row.split(":")[0])
        seconds_part = float(row.split(":")[1])
        minute_fraction = seconds_part / 60
        return (minutes_part +  minute_fraction)
    except:
        return '0'
jeremy_df['pace_fraction'] = jeremy_df['pace'].map(express_pace_decimal) 
jeremy_df

,name,date,distance,pace,duration,duration_sum,pace_fraction
0,Jeremy Bradford,2016-04-30 09:03:00,47.73 mi,15:39,12h 26m,746,15.650000
1,Jeremy Bradford,2016-04-29 21:01:00,4.17 mi,13:40,57m 0s,57,13.666667
2,Jeremy Bradford,2016-04-29 09:00:00,42.66 mi,12:02,8h 33m,513,12.033333
3,Jeremy Bradford,2016-04-28 10:02:00,36.34 mi,13:51,8h 23m,503,13.850000
4,Jeremy Bradford,2016-04-27 08:34:00,50.65 mi,13:26,11h 20m,680,13.433333
5,Jeremy Bradford,2016-04-26 10:20:00,40.42 mi,13:30,9h 5m,545,13.500000
6,Jeremy Bradford,2016-04-25 07:41:00,21.07 mi,13:16,4h 39m,279,13.266667
7,Jeremy Bradford,2016-04-23 21:29:00,34.78 mi,14:57,8h 40m,520,14.950000
8,Jeremy Bradford,2016-04-23 16:30:00,14.46 mi,12:27,2h 59m,179,12.450000
9,Jeremy Bradford,2016-04-22 17:18:00,36.24 mi,15:23,9h 17m,557,15.383333


In [45]:
#filter the data for run dates
jeremy_summary = jeremy_df.sort_values(by=['date'])
jeremy_summary = jeremy_summary[(jeremy_summary['date'] > '2016-04-16 17:09:00') & (jeremy_summary['date'] < '2016-06-23 06:07:0')]

jeremy_summary

,name,date,distance,pace,duration,duration_sum,pace_fraction
16,Jeremy Bradford,2016-04-18 07:32:00,47.78 mi,10:58,8h 44m,524,10.966667
15,Jeremy Bradford,2016-04-18 19:36:00,9.92 mi,14:20,2h 22m,142,14.333333
14,Jeremy Bradford,2016-04-19 06:57:00,60.16 mi,14:38,14h 40m,880,14.633333
13,Jeremy Bradford,2016-04-20 14:42:00,28.53 mi,14:57,7h 6m,426,14.950000
12,Jeremy Bradford,2016-04-21 09:19:00,21.07 mi,12:52,4h 31m,271,12.866667
11,Jeremy Bradford,2016-04-21 16:45:00,32.65 mi,13:14,7h 12m,432,13.233333
10,Jeremy Bradford,2016-04-22 10:59:00,14.02 mi,16:17,3h 48m,228,16.283333
9,Jeremy Bradford,2016-04-22 17:18:00,36.24 mi,15:23,9h 17m,557,15.383333
8,Jeremy Bradford,2016-04-23 16:30:00,14.46 mi,12:27,2h 59m,179,12.450000
7,Jeremy Bradford,2016-04-23 21:29:00,34.78 mi,14:57,8h 40m,520,14.950000


In [48]:
jeremy_summary['distance'] = jeremy_summary['distance'].str.replace('mi', '')
jeremy_summary

,name,date,distance,pace,duration,duration_sum,pace_fraction
16,Jeremy Bradford,2016-04-18 07:32:00,47.78,10:58,8h 44m,524,10.966667
15,Jeremy Bradford,2016-04-18 19:36:00,9.92,14:20,2h 22m,142,14.333333
14,Jeremy Bradford,2016-04-19 06:57:00,60.16,14:38,14h 40m,880,14.633333
13,Jeremy Bradford,2016-04-20 14:42:00,28.53,14:57,7h 6m,426,14.950000
12,Jeremy Bradford,2016-04-21 09:19:00,21.07,12:52,4h 31m,271,12.866667
11,Jeremy Bradford,2016-04-21 16:45:00,32.65,13:14,7h 12m,432,13.233333
10,Jeremy Bradford,2016-04-22 10:59:00,14.02,16:17,3h 48m,228,16.283333
9,Jeremy Bradford,2016-04-22 17:18:00,36.24,15:23,9h 17m,557,15.383333
8,Jeremy Bradford,2016-04-23 16:30:00,14.46,12:27,2h 59m,179,12.450000
7,Jeremy Bradford,2016-04-23 21:29:00,34.78,14:57,8h 40m,520,14.950000


In [50]:
#split dattime into 2 columns so I can grouby day
jeremy_summary['day'] = [d.date() for d in jeremy_summary['date']]
jeremy_summary['time'] = [d.time() for d in jeremy_summary['date']]

#convert to distance to number
jeremy_summary['distance']= pd.to_numeric(jeremy_summary['distance'], errors='coerce')
new = jeremy_summary.groupby(['day', 'name']).agg(['sum', 'mean'])[['distance', 'pace_fraction', 'duration_sum']].reset_index()
new.columns = new.columns.map('_'.join)
new.drop(['distance_mean', 'pace_fraction_sum', 'duration_sum_mean'], axis =1, inplace=True)
new.columns = ['day', 'name', 'total_distance', 'avg_pace', 'duration']
jeremy_run_summary = new
jeremy_run_summary

,day,name,total_distance,avg_pace,duration
0,2016-04-18,Jeremy Bradford,57.70,12.650000,666
1,2016-04-19,Jeremy Bradford,60.16,14.633333,880
2,2016-04-20,Jeremy Bradford,28.53,14.950000,426
3,2016-04-21,Jeremy Bradford,53.72,13.050000,703
4,2016-04-22,Jeremy Bradford,50.26,15.833333,785
5,2016-04-23,Jeremy Bradford,49.24,13.700000,699
6,2016-04-25,Jeremy Bradford,21.07,13.266667,279
7,2016-04-26,Jeremy Bradford,40.42,13.500000,545
8,2016-04-27,Jeremy Bradford,50.65,13.433333,680
9,2016-04-28,Jeremy Bradford,36.34,13.850000,503


In [51]:
jeremy_run_summary.to_csv('jeremy_summary.csv')